In [14]:
from src.tasks.infer import infer_from_trained

In [15]:
class D:
    pass
        
args = D()
args.model_no = 0
args.model_size = 'bert-base-uncased'
args.num_classes = 4 # TODO: Important: Change for nyt_mix_news to 4. NYT - 3

task = 'classification' # classification for proba / infer_one_sentence_labels

In [16]:
inferer = infer_from_trained(args, detect_entities=False, load_best=True, task=task)

03/16/2022 06:20:26 PM [INFO]: Loading tokenizer and model...
03/16/2022 06:20:27 PM [INFO]: loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home/roeel/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
03/16/2022 06:20:27 PM [INFO]: Model config {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pad_token_id": 0,
  "pruned_heads": {},
  "torchsc

Model config:  {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pad_token_id": 0,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 30522
}



03/16/2022 06:20:29 PM [INFO]: Weights of BertModel not initialized from pretrained model: ['bert.classification_layer.weight', 'bert.classification_layer.bias']
03/16/2022 06:20:30 PM [INFO]: Loaded best model.
03/16/2022 06:20:30 PM [INFO]: Loaded model and optimizer.
03/16/2022 06:20:30 PM [INFO]: Done!


## Extract Embeddings from BERT

In [17]:
import pandas as pd

def get_sent_embs(sent):
    inferer.net.task = 'fewrel'
    return inferer.get_one_sent_output(sent)[0].cpu().numpy()    

def get_all_embs(df,data_path,inferer):
    ddf = df
    if ddf is None:
        ddf = pd.read_pickle(data_path)
    
    ddf = ddf[ddf.sents.str.len() < inferer.tokenizer.max_len]
            
    ddf['emb'] = ddf.sents.apply(get_sent_embs)    
    return ddf

def convert_emb_to_df_cols(df):    
    new_df = pd.concat([df.drop(columns='emb'),pd.DataFrame(df.emb.tolist())],axis=1)
    cols = list(set(new_df.columns) - set(['sents', 'relations', 'relations_id']))
    return new_df, cols

In [18]:
df_test = get_all_embs(None, './data/df_test.pkl',inferer)
df_test,cols = convert_emb_to_df_cols(df_test.reset_index(drop=True))
df_train = get_all_embs(None, './data/df_train.pkl',inferer)
df_train,_ = convert_emb_to_df_cols(df_train.reset_index(drop=True))

## Find similar samples

In [19]:
import sys
sys.path += ['/home/dekelco/views/dsutil/']

In [20]:
from error_analysis.similar_samples import SimilarSamples

In [21]:
ss = SimilarSamples(df_train, columns=cols)

In [22]:
df_sim = ss.merge_similar_rows(df_test[20:30], k=2).reset_index()

In [23]:
s = df_sim.style
tt = pd.DataFrame(data=df_sim.sents_x.tolist(), columns=['sents_x'], index=df_sim.index)
s.set_tooltips(tt)
df_sim2 = df_sim.filter(regex=("index|sents|relations|relations_id"))
df_sim2.head(20)

,index,sents_x,relations_x,relations_id_x,sents_y,relations_y,relations_id_y
0,20,Minister of [E1]Cooperation and Development[/E...,/business/person/company,0,"Slovakian Minister of [E1]State[/E1] , [E2]Ing...",no_relation,3
1,20,Minister of [E1]Cooperation and Development[/E...,/business/person/company,0,Swiss Minister of [E1]State[/E1] in the Federa...,no_relation,3
2,24,Swiss Minister of State in [E1]the Federal Dep...,/business/person/company,0,Swiss Minister of [E1]State[/E1] in the Federa...,no_relation,3
3,21,Minister of [E1]State for International Cooper...,/business/person/company,0,NNA - UK Minister of [E1]State[/E1] for the Mi...,no_relation,3
4,24,Swiss Minister of State in [E1]the Federal Dep...,/business/person/company,0,NNA - UK Minister of [E1]State[/E1] for the Mi...,no_relation,3
5,21,Minister of [E1]State for International Cooper...,/business/person/company,0,"Moreover , Ambassador [E1]Nuland[/E1] stressed...",no_relation,3
6,22,[E1]The Norwegian Foreign Affairs[/E1] Ministe...,/business/person/company,0,"[E1]NNA[/E1] - President of the Republic , Gen...",no_relation,3
7,22,[E1]The Norwegian Foreign Affairs[/E1] Ministe...,/business/person/company,0,[E1]NNA[/E1] - UK Minister of State for the Mi...,no_relation,3
8,23,"[E1]The Dutch Foreign Affairs[/E1] Minister , ...",/business/person/company,0,"[E1]NNA[/E1] - The President of the Republic ,...",no_relation,3
9,23,"[E1]The Dutch Foreign Affairs[/E1] Minister , ...",/business/person/company,0,[E1]NNA -[/E1] The Lebanese Business leaders a...,no_relation,3


In [28]:
n=7
print(df_sim2.iloc[n].sents_x)
print()
print(df_sim2.iloc[n].sents_y)

[E1]The Norwegian Foreign Affairs[/E1] Minister , [E2]Mary Ericson Soreed[/E2] , stressed that the Beirut explosion , took place while Lebanon passes through a very deep crisis " And since about a year until today , Lebanon has been without a government and without knowing the causes of the explosion " .

[E1]NNA[/E1] - UK Minister of State for the Middle East and [E2]International Development[/E2] , Alistair Burt , has reaffirmed the UK 's commitment to supporting a strong and prosperous Lebanon , announcing further support to the Lebanese Army and celebrating the UK 's investment in Lebanon 's future .


## Predict single sample

In [30]:
sent = "The CEO of [E1] IBM [\E1] , [E2] Steve Cohen [\E2] , said he will discuss it with the partners"
inferer.net.task = 'classification'
sent = sent.replace('[\\','[/')
preds = inferer.infer_one_sentence_labels(sent)

emb = get_sent_embs(sent)
df_single = pd.DataFrame(data=[sent],columns=['sents'])
df_single = pd.concat([df_single,pd.DataFrame(data=[emb.tolist()])],axis=1)


df_sim3 = ss.merge_similar_rows(df_single, k=2).reset_index()
df_sim3 = df_sim3.filter(regex=("index|sents|relations|relations_id"))
print('**sentence**')
print(df_sim3.iloc[0].sents_x)
print('\n**most similar**')
print(df_sim3.iloc[0].sents_y)
print(f'\n**similar class={df_sim3.iloc[0].relations}**')
print('\n2nd most similar\n----------------')
print(df_sim3.iloc[1].sents_y)
print(f'\n**2nd similar class={df_sim3.iloc[1].relations}**')

print(f'\n**Relation prediction**')
preds


**sentence**
The CEO of [E1] IBM [/E1] , [E2] Steve Cohen [/E2] , said he will discuss it with the partners

**most similar**
It is the book 's interpretation of the facts that can be disputed , said Thitinan Pongsudhirak , the director of the Institute of Security and International Studies at [E2]  Chulalongkorn University[/E2] in [E1]  Bangkok[/E1] .

**similar class=/location/location/contains**

2nd most similar
----------------
METRO TOUR SERVICE Today and tomorrow at 2 p.m. , visits to [E1]  Brooklyn[/E1] Heights , [E2]  Fort Greene[/E2] and Clinton Hill , Brooklyn .

**2nd similar class=/location/location/contains**

**Relation prediction**


[[{'score': 9.740673704072833e-05, 'label': '/business/person/company'},
  {'score': 0.9775983095169067, 'label': '/location/location/contains'},
  {'score': 0.00034371347283013165, 'label': '/people/person/place_lived'},
  {'score': 0.021960614249110222, 'label': 'no_relation'}]]

In [14]:
df_sim3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   index         1 non-null      int64 
 1   sents_x       1 non-null      object
 2   sents_y       1 non-null      object
 3   relations     1 non-null      object
 4   relations_id  1 non-null      int64 
dtypes: int64(2), object(3)
memory usage: 168.0+ bytes


### Spacy

In [ ]:
import spacy
nlp = spacy.load('en_core_web_trf')

In [ ]:
sent = "European Investment Bank President ,Warner Heuer"
sents_doc = nlp(sent)
ents = sents_doc.ents # get entities
for ent in ents:
    print(ent.label_, ent.text)